### Neste notebook, coletamos e salvamos em .csv os municípios do Ceará com os respectivos códigos do IBGE e as regionais de saúde do Estado às quais eles pertencem

In [36]:
import pandas as pd

In [60]:
df_municipios = pd.read_json('https://integrasus.saude.ce.gov.br/api/cobertura-vacinal-covid/cobertura-vacinal-mapa-bivalente?dataInicio=2023-02-27&dataFim=2023-07-12&tipoVacina=Bivalente&casos=&superintendencia=&ads=&municipio=&dose=Dose%201&indicadores=')
df_municipios = df_municipios[['codMunicipio', 'municipio']]

In [61]:
df_regionais = pd.read_json('https://integrasus.saude.ce.gov.br/api/cobertura-vacinal-covid/filter-superintendencia?indicadores=&dose=Dose%201&municipio=&ads=&superintendencia=&tipoVacina=Bivalente&dataFim=2023-07-12&dataInicio=2023-02-27')

In [62]:
df_regionais = df_regionais.apply(lambda x: x.str.replace(' ', '%20').str.replace('Ã', '%C3%83'))

In [63]:
df_regionais

,superintendencia
0,CARIRI
1,FORTALEZA
2,LITORAL%20LESTE/JAGUARIBE
3,SERT%C3%83O%20CENTRAL
4,SOBRAL


In [64]:
regioes = df_regionais['superintendencia'].to_list()
regioes

['CARIRI',
 'FORTALEZA',
 'LITORAL%20LESTE/JAGUARIBE',
 'SERT%C3%83O%20CENTRAL',
 'SOBRAL']

In [65]:
lista_regionais = []
for regiao in regioes:
    df_regiao = pd.read_json(f'https://integrasus.saude.ce.gov.br/api/cobertura-vacinal-covid/filter-municipio?ads=&superintendencia={regiao}&tipoVacina=Bivalente&dataFim=2023-07-12&dataInicio=2023-02-27')
    df_regiao['regiao'] = regiao
    lista_regionais.append(df_regiao)

In [66]:
df_municipios_regionais = pd.concat(lista_regionais)

In [67]:
df_municipios_regionais = df_municipios_regionais.apply(lambda x: x.str.replace('%20', ' ').str.replace('SERT%C3%83', 'SERTÃ'))

In [68]:
df_municipios_regionais['regiao'].unique()

array(['CARIRI', 'FORTALEZA', 'LITORAL LESTE/JAGUARIBE', 'SERTÃO CENTRAL',
       'SOBRAL'], dtype=object)

In [69]:
df_final = df_municipios.merge(df_municipios_regionais, on='municipio')

In [70]:
df_final.to_csv('df_municipios_regionais.csv', index=False)